# Objective:
  In this project, you will gain the opportunity to explore different LLM agents (Judge, Jury, Witness, Prosecution, and Defense) interacting with each other to defend and support popular court cases.

## Directions (Estimated Time 30 Min):
  1. Breakout into groups (up to 3) and run through each scenario provided in the Google Colab
  2. Read each case carefully, note any reasoning patterns you notice – or anything you did/did not expect
  3. Answer the following questions at the bottom of the docs
  4. Time permitting: Brainstorm your own scenario and see what the agents output
  5. Come back to larger team and open discussion


In [ ]:
#These will show you the directions

# Case 1: FBI vs. Apple

In [ ]:
"""
Single‑model courtroom simulation (FBI vs Apple) where **every role** is powered by
OpenAI ChatGPT and the jurors hold a two‑round conversation before voting.
"""
from __future__ import annotations

import os
import openai
from typing import List, Dict

# ───────────────────────────────────────── CONSTANTS ──────────────────────────────────────────
OPENAI_API_KEY: str = os.getenv("OPENAI_API_KEY", "INSERT API KEY HERE")
MODEL_NAME: str = os.getenv("CHATGPT_MODEL", "gpt-4o")  # Default to gpt-4o; override with env var  # fast & cheap – swap in gpt‑4o for higher quality
TEMPERATURE: float = 0.8
ROUNDS: int = 2          # how many jury rounds before the silent ballot

CASE_CONTEXT = """
Case: FBI vs. Apple Inc. (2016)
Summary:
Following the San Bernardino terrorist attack, the FBI recovered an iPhone used by
one of the shooters. Unable to access it due to encryption, the FBI demanded Apple
develop a backdoor to unlock the device. Apple refused, citing risks to global
cyber‑security and user privacy. The case presents a constitutional and ethical
conflict between public safety and digital privacy.
"""

PROSECUTOR_PROMPT = f"""
You are the lead prosecutor representing the FBI in the case FBI vs. Apple Inc.
Argue that Apple must comply with the court order to unlock the iPhone. Use legal,
ethical, and national‑security justifications.
{CASE_CONTEXT}
"""

DEFENSE_PROMPT = f"""
You are the defense attorney for Apple Inc., opposing the FBI's request. Argue that
building a backdoor undermines digital privacy, sets a dangerous precedent, and
could be abused globally. Use constitutional, technical, and ethical reasoning.
{CASE_CONTEXT}
"""

WITNESS_PROMPT = f"""
You are a sworn expert witness in the FBI vs. Apple case. Choose one role:
· A cybersecurity researcher
· An Apple engineer
· A constitutional scholar
Provide neutral, technical, or legal testimony on the risks/benefits of creating a
backdoor for law enforcement.
{CASE_CONTEXT}
"""

JUDGE_TEMPLATE = """
You are the judge presiding over FBI vs. Apple. Evaluate the FBI’s argument,
Apple’s defense, and the expert witness testimony.

FBI (Prosecution):\n{prosecution}\n\nApple (Defense):\n{defense}\n\nExpert Witness:\n{witness}\n\nProvide a concise judicial summary weighing the legal and constitutional merit of
each side.
"""

JURORS: List[Dict[str, str]] = [
    {"name": "Legalist AI",  "style": "You strictly follow statute and precedent."},
    {"name": "Empath AI",    "style": "You emphasize digital rights and human impact."},
    {"name": "Utilitarian AI","style": "You weigh societal costs and benefits."},
    {"name": "Data AI",      "style": "You focus on technical feasibility and risk."},
    {"name": "Justice AI",   "style": "You guard against power abuse and inequality."},
    {"name": "Skeptic AI",   "style": "You demand hard evidence and challenge claims."},
]

JUROR_TEMPLATE = """
You are {name}, a juror in FBI vs. Apple.
{style}

Arguments so far:
• FBI: {prosecution}
• Apple: {defense}
• Expert: {witness}

Conversation history among jurors (most recent first):
{history}

Deliberate out loud during round {round_num}/{total_rounds}. Address other jurors by
name, question their premises, defend your own, and propose compromises. **Do not**
yet cast a final vote. End with a short summary of where you currently lean ("lean
Guilty" vs "lean Not Guilty" or "undecided").
"""

FINAL_VOTE_TEMPLATE = """
You are {name}, a juror in FBI vs. Apple.
Here is the entire conversation so far:
{full_history}

Time to cast your vote. Decide: "Guilty" (Apple must comply) or "Not Guilty" (Apple
is justified in refusal). Provide <50 words of reasoning then state your single‑word
verdict on the last line alone.
"""

# ───────────────────────────────────────── OPENAI WRAPPER ─────────────────────────────────────

def chatgpt_call(prompt: str) -> str:
    """Wrapper with simple fallback cascade if selected MODEL_NAME is unavailable."""
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    tried: List[str] = []
    for model in [MODEL_NAME, "gpt-4o", "gpt-3.5-turbo"]:
        if model in tried:
            continue
        tried.append(model)
        try:
            resp = client.chat.completions.create(
                model=model,
                temperature=TEMPERATURE,
                messages=[{"role": "user", "content": prompt}],
            )
            return resp.choices[0].message.content.strip()
        except openai.BadRequestError as e:
            # Only retry on invalid model ID; raise others.
            if "invalid model" not in str(e):
                raise
    raise RuntimeError(f"None of the fallback models {{{', '.join(tried)}}} are available to this key.")

# ───────────────────────────────────────── MAIN SIMULATION ────────────────────────────────────

def simulate() -> None:
    print("\n=== FBI vs. Apple: ChatGPT‑Only Simulation ===\n")

    # --- Core arguments ---
    prosecution = chatgpt_call(PROSECUTOR_PROMPT)
    defense     = chatgpt_call(DEFENSE_PROMPT)
    witness     = chatgpt_call(WITNESS_PROMPT)

    print("Prosecution (FBI):\n", prosecution, "\n")
    print("Defense (Apple):\n", defense, "\n")
    print("Expert Witness:\n", witness, "\n")

    judge_summary = chatgpt_call(JUDGE_TEMPLATE.format(
        prosecution=prosecution, defense=defense, witness=witness))
    print("=== Judge’s Charge ===\n", judge_summary, "\n")

    # --- Jury deliberation ---
    conversation: List[str] = []  # store strings like "Legalist AI: ..."

    for rnd in range(1, ROUNDS + 1):
        print(f"\n--- Jury Round {rnd}/{ROUNDS} ---")
        for juror in JURORS:
            history_snippet = "\n".join(conversation[-6:]) or "(no prior discussion)"
            juror_text = chatgpt_call(JUROR_TEMPLATE.format(
                name=juror["name"], style=juror["style"],
                prosecution=prosecution, defense=defense, witness=witness,
                history=history_snippet, round_num=rnd, total_rounds=ROUNDS))
            tag = f"{juror['name']}: "
            conversation.append(tag + juror_text)
            print(f"\n{tag}{juror_text}\n")

    # --- Final silent ballot ---
    print("\n=== Final Votes ===")
    votes: Dict[str, str] = {}
    for juror in JURORS:
        full_hist = "\n".join(conversation)
        vote_text = chatgpt_call(FINAL_VOTE_TEMPLATE.format(
            name=juror["name"], full_history=full_hist))
        verdict = "Not Guilty" if "Not Guilty" in vote_text else ("Guilty" if "Guilty" in vote_text else "Abstain")
        votes[juror["name"]] = verdict
        print(f"{juror['name']}: {verdict}")

    guilty = list(votes.values()).count("Guilty")
    not_guilty = list(votes.values()).count("Not Guilty")
    abstain = list(votes.values()).count("Abstain")

    print(f"\nTally → Guilty: {guilty}, Not Guilty: {not_guilty}, Abstain: {abstain}")
    if guilty > len(JURORS) / 2:
        print("🧑‍⚖️ Verdict: APPLE MUST COMPLY")
    elif not_guilty > len(JURORS) / 2:
        print("🧑‍⚖️ Verdict: APPLE IS JUSTIFIED")
    else:
        print("🧑‍⚖️ Verdict: HUNG JURY")


# ───────────────────────────────────────── ENTRY POINT ─────────────────────────────────────────
if __name__ == "__main__":
    simulate()




=== FBI vs. Apple: ChatGPT‑Only Simulation ===

Prosecution (FBI):
 Your Honor, esteemed members of the court, today we deliberate on a matter of paramount importance not only to national security but also to our collective responsibility to uphold justice and protect innocent lives. The case of FBI vs. Apple Inc. revolves around a critical question: Should a corporation’s commitment to consumer privacy outweigh the imperative to safeguard national security and ensure public safety? We posit that Apple must comply with the court order to unlock the iPhone in question, and we present our arguments grounded in legal, ethical, and national-security considerations.

**Legal Justifications:**

1. **All Writs Act of 1789:** The All Writs Act empowers courts to issue orders necessary to assist in their duties. The statute provides the legal foundation for compelling third parties to aid law enforcement in executing their responsibilities, provided the request is reasonable and necessary. In 

# Case 2: Oregon vs BrightRiver

In [ ]:
"""
Single-model courtroom simulation (Oregon vs BrightRiver) where **every role**
is powered by OpenAI ChatGPT and the jurors hold a two-round conversation
before voting.
"""
from __future__ import annotations

import os, openai
from typing import List, Dict

# ────────────────────────────────── CONSTANTS ──────────────────────────────────
OPENAI_API_KEY: str = os.getenv("OPENAI_API_KEY", "INSERT API KEY HERE")
MODEL_NAME: str      = os.getenv("CHATGPT_MODEL", "gpt-4o")
TEMPERATURE: float   = 1.1
ROUNDS: int          = 2          # jury discussion rounds

CASE_CONTEXT = """
Case: State of Oregon v. BrightRiver AI Water-Management (2025)

Facts
• BrightRiver sells an AI service that predicts optimal times for farmers
  to pump water from the Columbia River, balancing crop yield and salmon
  spawning protections.
• During a historic heatwave, hundreds of farms followed BrightRiver’s
  forecast and drew water 11 p.m.–3 a.m. for three nights.
• A massive die-off of endangered sockeye salmon followed. Biologists say
  late-night pumping was a *significant contributing factor*, but heat stress
  and an upstream chemical spill also played roles.
• Oregon is charging BrightRiver with “reckless endangerment of protected
  species” and seeks to enjoin its service during spawning season.

Foggy Points
1. BrightRiver’s licence agreement warns forecasts are “advisory only.”
2. The state’s own water-allocation portal was down, leaving farmers to rely
   on BrightRiver as the de-facto authority.
3. BrightRiver’s model flagged a 14 % ecological-harm risk—below its unilateral
   20 % alert threshold.
"""

PROSECUTOR_PROMPT = f"""
You are a state prosecutor. Argue that BrightRiver acted recklessly by pushing
a pumping schedule that endangered an already-stressed endangered species.
Highlight the duty of care under wildlife-protection statutes and the fact that
BrightRiver effectively displaced official guidance.
{CASE_CONTEXT}
"""

DEFENSE_PROMPT = f"""
You represent BrightRiver. Argue that (1) the salmon die-off was multi-causal
(heatwave + industrial spill), (2) BrightRiver clearly disclaimed coercive power,
and (3) shutting the service will bankrupt small farmers who now lack any
predictive tool.
{CASE_CONTEXT}
"""

WITNESS_PROMPT = f"""
You are an expert witness—choose ONE:
· Computational ecologist
· Agricultural economist
· AI-governance scholar
Offer neutral testimony on BrightRiver’s 20 % harm threshold, alternative
safeguards, and how responsibility should be shared among BrightRiver, farmers,
and the state agency.
{CASE_CONTEXT}
"""

JUDGE_TEMPLATE = """
You are the judge presiding over Oregon vs BrightRiver. Evaluate prosecution,
defense, and the expert witness testimony.

Prosecution:\n{prosecution}\n\nDefense:\n{defense}\n\nExpert Witness:\n{witness}

Provide a concise judicial summary weighing the legal and constitutional merit
of each side.
"""

JURORS: List[Dict[str, str]] = [
    {"name": "Legalist AI",  "style": "Strictly follows statute and precedent."},
    {"name": "Empath AI",    "style": "Prioritises human impact and moral rights."},
    {"name": "Utilitarian AI","style": "Weighs societal costs and benefits."},
    {"name": "Data AI",      "style": "Focuses on causal data and technical risk."},
    {"name": "Justice AI",   "style": "Guards against corporate power abuses."},
    {"name": "Skeptic AI",   "style": "Demands hard evidence; challenges claims."},
]

JUROR_TEMPLATE = """
You are {name}, a juror in Oregon vs BrightRiver.
{style}

Arguments so far:
• Prosecutor: {prosecution}
• Defense: {defense}
• Expert: {witness}

Most-recent juror comments:
{history}

Speak during round {round_num}/{total_rounds}. Address other jurors, challenge
points, defend your view, but **do not vote yet**. End with your current lean
("lean Guilty", "lean Not Guilty", or "undecided").
"""

FINAL_VOTE_TEMPLATE = """
You are {name}, a juror in Oregon vs BrightRiver.
Here is the full jury discussion:
{full_history}

Cast your final vote: "Guilty" (BrightRiver liable) or "Not Guilty"
(BrightRiver justified). Give <50 words of reasoning, then put your one-word
verdict on the last line alone.
"""

# ───────────────────────────── OPENAI WRAPPER ────────────────────────────────
def chatgpt_call(prompt: str) -> str:
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    for model in (MODEL_NAME, "gpt-4o", "gpt-3.5-turbo"):
        try:
            resp = client.chat.completions.create(
                model=model,
                temperature=TEMPERATURE,
                messages=[{"role": "user", "content": prompt}],
            )
            return resp.choices[0].message.content.strip()
        except openai.BadRequestError as e:
            if "invalid model" in str(e):
                continue
            raise
    raise RuntimeError("No usable model available for this API key.")

# ───────────────────────────── MAIN SIMULATION ───────────────────────────────
def simulate() -> None:
    print("\n=== Oregon vs BrightRiver: ChatGPT-Only Simulation ===\n")

    prosecution = chatgpt_call(PROSECUTOR_PROMPT)
    defense     = chatgpt_call(DEFENSE_PROMPT)
    witness     = chatgpt_call(WITNESS_PROMPT)

    print("Prosecution:\n", prosecution, "\n")
    print("Defense:\n", defense, "\n")
    print("Expert Witness:\n", witness, "\n")

    judge_summary = chatgpt_call(JUDGE_TEMPLATE.format(
        prosecution=prosecution, defense=defense, witness=witness))
    print("=== Judge’s Charge ===\n", judge_summary, "\n")

    conversation: List[str] = []
    for rnd in range(1, ROUNDS + 1):
        print(f"\n--- Jury Round {rnd}/{ROUNDS} ---")
        for juror in JURORS:
            snippet = "\n".join(conversation[-6:]) or "(no prior discussion)"
            speech  = chatgpt_call(JUROR_TEMPLATE.format(
                name=juror["name"], style=juror["style"],
                prosecution=prosecution, defense=defense, witness=witness,
                history=snippet, round_num=rnd, total_rounds=ROUNDS))
            tag = f"{juror['name']}: "
            conversation.append(tag + speech)
            print(f"\n{tag}{speech}\n")

    print("\n=== Final Votes ===")
    votes: Dict[str, str] = {}
    full_hist = "\n".join(conversation)
    for juror in JURORS:
        vote   = chatgpt_call(FINAL_VOTE_TEMPLATE.format(
                 name=juror["name"], full_history=full_hist))
        verdict = ("Not Guilty" if "Not Guilty" in vote else
                   "Guilty" if "Guilty" in vote else "Abstain")
        votes[juror["name"]] = verdict
        print(f"{juror['name']}: {verdict}")

    g, ng = votes.values().count("Guilty"), votes.values().count("Not Guilty")
    ab    = votes.values().count("Abstain")
    print(f"\nTally → Guilty: {g}, Not Guilty: {ng}, Abstain: {ab}")
    if g  > len(JURORS)/2: print("🧑‍⚖️ Verdict: BRIGHTRIVER LIABLE")
    elif ng > len(JURORS)/2: print("🧑‍⚖️ Verdict: BRIGHTRIVER JUSTIFIED")
    else: print("🧑‍⚖️ Verdict: HUNG JURY")

# ───────────────────────────── ENTRY POINT ───────────────────────────────────
if __name__ == "__main__":
    simulate()



=== Oregon vs BrightRiver: ChatGPT-Only Simulation ===

Prosecution:
 Members of the court, the case before you today involves a matter of significant ecological and legal importance. In State of Oregon v. BrightRiver AI Water-Management, the defendant stands accused of reckless endangerment of a protected species due to its negligent conduct during one of Oregon's most critical environmental periods.

BrightRiver, by providing an AI service intended to optimize water usage for agriculture, entered an area where significant duties of care must prevail over mere profitability or efficiency objectives. The state acknowledges that their predictions are intended to balance necessary agricultural needs with the vital ecological sanctity of endangered salmon species, such as the sockeye salmon. However, it is precisely through BrightRiver's failure to uphold these responsibilities that they acted with reckless disregard for the same endangered species they purported to protect.

First and f

AttributeError: 'dict_values' object has no attribute 'count'

# Case 3: City of Chicago v. Nexus Mobility Autonomous Shuttle



In [ ]:
"""
"""
from __future__ import annotations

import os, openai
from typing import List, Dict

# ─────────────────────────────────── CONSTANTS ───────────────────────────────────
OPENAI_API_KEY: str = os.getenv(
    "OPENAI_API_KEY",
    "INSERT API KEY HERE"
)  # <--- insert your secret if not using env-var
MODEL_NAME: str    = os.getenv("CHATGPT_MODEL", "gpt-4o")
TEMPERATURE: float = 1.1
ROUNDS: int        = 2  # jury discussion rounds

CASE_CONTEXT = """
Case: City of Chicago v. Nexus Mobility Autonomous Shuttle (2026)

Facts
• Nexus operates driver-less shuttles on fixed downtown routes, mainly serving
  elderly and disabled riders.
• At 10:43 p.m. on 14 May 2026 a shuttle struck an 82-year-old pedestrian who
  crossed against the light at a construction detour. The victim later died.
• Dash-cam shows the pedestrian emerged from behind a dumpster; LIDAR returns
  briefly dropped out because scaffold-mesh reflected the beam.
• City engineers had removed audible crossing beacons and left temporary
  signage half-covered during roadworks.
• A software update that improved glare-robust object tracking passed QA but
  was postponed to avoid rush-hour downtime.

Charges
Chicago seeks a criminal-negligence conviction and immediate suspension of the
Nexus permit.

Foggy Points
1.  The victim jaywalked and ignored a visual “Don’t Walk” signal (no audible cue).
2.  Nexus met — barely — the state’s 2019 autonomy-safety regs, which many experts
    call outdated.
3.  The city’s construction contractor deviated from its own traffic-control plan.
4.  Nexus’s risk log predicted a 0.7 % annual fatality rate — lower than human-
    driven city buses (0.9 %) — and accepted that level without external review.
"""

PROSECUTOR_PROMPT = f"""
You are an assistant state’s attorney.  Argue that Nexus acted with criminal
negligence: knowing its LIDAR was vulnerable to scaffold reflections, it delayed
a safety patch and failed to add redundant thermal sensors, effectively treating
deaths as an “acceptable” cost.  Stress the heightened duty of care owed to
vulnerable road users.
{CASE_CONTEXT}
"""

DEFENSE_PROMPT = f"""
You represent Nexus Mobility.  Argue that (1) the victim unlawfully entered the
roadway, (2) city contractors removed mandated safety beacons, creating an
unmapped hazard, and (3) Nexus shuttles still outperform human drivers on
fatality metrics; a shutdown would strand thousands of disabled riders who rely
on the service.
{CASE_CONTEXT}
"""

WITNESS_PROMPT = f"""
You are an independent expert witness — choose ONE:
· Autonomous-vehicle sensor engineer
· Urban road-safety researcher
· Disability-rights advocate

Offer neutral testimony on whether Nexus met industry safety norms, how much
the detour contributed, and what mitigation options were feasible.
{CASE_CONTEXT}
"""

JUDGE_TEMPLATE = """
You are the judge presiding over Chicago vs Nexus.  Evaluate the prosecution,
defense, and expert witness testimony.

Prosecution:
{prosecution}

Defense:
{defense}

Expert Witness:
{witness}

Provide a concise judicial summary weighing the legal and constitutional merit
of each side.
"""

JURORS: List[Dict[str, str]] = [
    {"name": "Legalist AI",  "style": "Strictly follows statute and precedent."},
    {"name": "Empath AI",    "style": "Prioritises human impact and moral rights."},
    {"name": "Utilitarian AI","style": "Weighs societal costs and benefits."},
    {"name": "Data AI",      "style": "Focuses on causal data and technical risk."},
    {"name": "Justice AI",   "style": "Guards against corporate power abuses."},
    {"name": "Skeptic AI",   "style": "Demands hard evidence; challenges claims."},
]

JUROR_TEMPLATE = """
You are {name}, a juror in Chicago vs Nexus.
{style}

Arguments so far:
• Prosecutor: {prosecution}
• Defense:    {defense}
• Expert:     {witness}

Most-recent juror comments:
{history}

Speak during round {round_num}/{total_rounds}. Address other jurors, challenge
points, defend your view, but **do not vote yet**. End with your current lean —
“lean Guilty”, “lean Not Guilty”, or “undecided”.
"""

FINAL_VOTE_TEMPLATE = """
You are {name}, a juror in Chicago vs Nexus.
Here is the full jury discussion:
{full_history}

Cast your final vote: “Guilty” (Nexus liable) **or** “Not Guilty” (Nexus
justified).  Give <50 words of reasoning, then put your one-word verdict on the
last line alone.
"""

# ───────────────────────────────── OPENAI WRAPPER ──────────────────────────────
def chatgpt_call(prompt: str) -> str:
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    for model in (MODEL_NAME, "gpt-4o", "gpt-3.5-turbo"):
        try:
            resp = client.chat.completions.create(
                model=model,
                temperature=TEMPERATURE,
                messages=[{"role": "user", "content": prompt}],
            )
            return resp.choices[0].message.content.strip()
        except openai.BadRequestError as e:
            if "invalid model" in str(e):
                continue
            raise
    raise RuntimeError("No usable model available for this API key.")

# ───────────────────────────────── MAIN SIMULATION ─────────────────────────────
def simulate() -> None:
    print("\n=== Chicago vs Nexus: ChatGPT-Only Simulation ===\n")

    prosecution = chatgpt_call(PROSECUTOR_PROMPT)
    defense     = chatgpt_call(DEFENSE_PROMPT)
    witness     = chatgpt_call(WITNESS_PROMPT)

    print("Prosecution:\n", prosecution, "\n")
    print("Defense:\n",   defense,     "\n")
    print("Expert Witness:\n", witness, "\n")

    judge_summary = chatgpt_call(JUDGE_TEMPLATE.format(
        prosecution=prosecution, defense=defense, witness=witness))
    print("=== Judge’s Charge ===\n", judge_summary, "\n")

    conversation: List[str] = []
    for rnd in range(1, ROUNDS + 1):
        print(f"\n--- Jury Round {rnd}/{ROUNDS} ---")
        for juror in JURORS:
            snippet = "\n".join(conversation[-6:]) or "(no prior discussion)"
            speech  = chatgpt_call(JUROR_TEMPLATE.format(
                name=juror["name"], style=juror["style"],
                prosecution=prosecution, defense=defense, witness=witness,
                history=snippet, round_num=rnd, total_rounds=ROUNDS))
            tag = f"{juror['name']}: "
            conversation.append(tag + speech)
            print(f"\n{tag}{speech}\n")

    print("\n=== Final Votes ===")
    votes: Dict[str, str] = {}
    full_hist = "\n".join(conversation)
    for juror in JURORS:
        vote_text = chatgpt_call(FINAL_VOTE_TEMPLATE.format(
                      name=juror["name"], full_history=full_hist))
        verdict = ("Not Guilty" if "Not Guilty" in vote_text
                   else "Guilty" if "Guilty" in vote_text
                   else "Abstain")
        votes[juror["name"]] = verdict
        print(f"{juror['name']}: {verdict}")

    g  = list(votes.values()).count("Guilty")
    ng = list(votes.values()).count("Not Guilty")
    ab = list(votes.values()).count("Abstain")
    print(f"\nTally → Guilty: {g}, Not Guilty: {ng}, Abstain: {ab}")
    if g > len(JURORS)/2:
        print("🧑‍⚖️ Verdict: NEXUS LIABLE")
    elif ng > len(JURORS)/2:
        print("🧑‍⚖️ Verdict: NEXUS JUSTIFIED")
    else:
        print("🧑‍⚖️ Verdict: HUNG JURY")

# ───────────────────────────────── ENTRY POINT ────────────────────────────────
if __name__ == "__main__":
    simulate()



=== Chicago vs Nexus: ChatGPT-Only Simulation ===

Prosecution:
 In the case of City of Chicago v. Nexus Mobility Autonomous Shuttle, strong arguments can be made that Nexus acted with criminal negligence, even amidst the complexities of autonomous vehicle regulations and the events surrounding the unfortunate incident.

1. **Breach of Duty of Care**: Nexus, by operating autonomous shuttles serving a population consisting predominantly of elderly and disabled riders, assumed a heightened duty of care. Vulnerable road users, such as the elderly and disabled, deserve increased protection. Nexus's operation within Chicago's dense urban environment further amplifies this duty, requiring meticulous foresight and proactive safeguards against foreseeable risks specific to urban vehicular operations.

2. **Knowledge of Vulnerabilities**: Nexus had knowledge, through its own operational risk assessments, that the LIDAR system employed in its shuttles was vulnerable to distortions caused by ref

# Post-Case Review

1. Which of the three cases surprised you the most? What was most expected?

2. How close do you believe this mimics human thinking patterns? Is it getting closer?

3. 	What single additional fact (a document, a sensor log, a policy memo) would have flipped your vote in any one case—and why?

4. Would you rely on Multi-Agent AI to be just and fair in court cases? Where is the line drawn if so?

# Case 4: Design Your Own!

In [ ]:
# ======================  FILL-IN-THE-BLANK CASE & RUN  ======================
# Edit INSIDE the <<< >>> markers, then run the cell once.  Done.

from __future__ import annotations
import os, openai, textwrap
from typing import List, Dict

# --- REQUIRED: your OpenAI key ---
OPENAI_API_KEY = "sk-REPLACE_ME"

# ==========  <<<  YOUR CASE STARTS HERE  >>>  ==========
CASE_NAME = "<<< Case title – e.g., Republic of Mars v. Terraform Corp (2031) >>>"

FACTS: List[str] = [
    "<<< Fact #1 >>>",
    "<<< Fact #2 >>>",
    "<<< Fact #3 (add or remove rows) >>>",
]

MAIN_CHARGE = "<<< e.g., criminal negligence & call for immediate shutdown >>>"

AMBIGUITIES: List[str] = [
    "<<< Ambiguity #1 – what the jury won’t know for sure >>>",
    "<<< Ambiguity #2 >>>",
]

EXPERT_ROLES: List[str] = [
    "<<< Role 1 – e.g., planetary geologist >>>",
    "<<< Role 2 – e.g., environmental economist >>>",
]
# ==========  <<<  YOUR CASE ENDS HERE  >>>  ==========


# ───────────────────  DO NOT EDIT BELOW THIS LINE  ─────────────────── #

MODEL = "gpt-4o"          # fallback chain inside wrapper
TEMP  = 1.0               # spicier debate
ROUNDS = 2                # jury-discussion rounds

def bullets(items: List[str], numbered=False) -> str:
    if numbered:
        return "\n".join(f"{i+1}. {x}" for i, x in enumerate(items, 1))
    return "\n".join(f"• {x}" for x in items)

CASE_CONTEXT = textwrap.dedent(f"""
Case: {CASE_NAME}

Facts
{bullets(FACTS)}

Charges
{MAIN_CHARGE}

Foggy Points
{bullets(AMBIGUITIES, numbered=True)}
""").strip()

PROSECUTOR = ("You are the lead prosecutor.  Summarise the facts and argue why "
              "the defendant is liable/negligent, emphasising statutory duty of "
              "care and public interest.\n{context}")
DEFENSE = ("You represent the defendant.  Present a robust defence, highlighting "
           "alternative causes, economic impact of shutdown, and any exculpatory data.\n{context}")
WITNESS = ("You are an expert witness — choose ONE of: "
           + ", ".join(EXPERT_ROLES)
           + ".  Provide neutral testimony clarifying disputed points.\n{context}")

JURORS: List[Dict[str, str]] = [
    {"name": "Legalist AI",  "style": "Strictly follows statute and precedent."},
    {"name": "Empath AI",    "style": "Prioritises human impact and moral rights."},
    {"name": "Utilitarian AI","style": "Weighs societal costs and benefits."},
    {"name": "Data AI",      "style": "Focuses on causal data and technical risk."},
    {"name": "Justice AI",   "style": "Guards against corporate power abuses."},
    {"name": "Skeptic AI",   "style": "Demands hard evidence; challenges claims."},
]

def chat(prompt: str, temp=TEMP) -> str:
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    for model in (MODEL, "gpt-3.5-turbo"):
        try:
            r = client.chat.completions.create(
                model=model, temperature=temp,
                messages=[{"role": "user", "content": prompt}]
            )
            return r.choices[0].message.content.strip()
        except openai.BadRequestError as e:
            if "invalid model" in str(e):
                continue
            raise
    raise RuntimeError("No authorised model for this key.")

# ---------------  Build prompts ---------------
prosecution = chat(PROSECUTOR.format(context=CASE_CONTEXT))
defense     = chat(DEFENSE.format(context=CASE_CONTEXT))
witness     = chat(WITNESS.format(context=CASE_CONTEXT))

JUDGE_TMPL = """
You are the judge. Evaluate prosecution, defense, and witness testimony.

Prosecution:
{pro}

Defense:
{def_}

Expert Witness:
{wit}

Provide a concise judicial summary weighing legal and constitutional merits.
"""
judge = chat(JUDGE_TMPL.format(pro=prosecution, def_=defense, wit=witness))

print("\n=== Prosecution ===\n", prosecution,
      "\n\n=== Defense ===\n", defense,
      "\n\n=== Expert Witness ===\n", witness,
      "\n\n=== Judge’s Charge ===\n", judge)

# ---------------  Jury ---------------
history: List[str] = []
JUROR_TMPL = """
You are {name}, a juror. {style}

Arguments so far:
• Prosecutor: {pro}
• Defense:    {def_}
• Expert:     {wit}

Last comments:
{hist}

Speak during round {r}/{R}. Address other jurors, but **do not vote yet**.
End with your lean: 'lean Guilty', 'lean Not Guilty', or 'undecided'.
"""
VOTE_TMPL = """
You are {name}, a juror. Here is the full discussion so far:
{full}

Cast your silent vote. Give <50 words reasoning, then on the last line write only:
Guilty   or   Not Guilty
"""

for rnd in range(1, ROUNDS + 1):
    print(f"\n--- Jury Round {rnd}/{ROUNDS} ---")
    for juror in JURORS:
        snippet = "\n".join(history[-6:]) or "(no prior comments)"
        speech = chat(JUROR_TMPL.format(
            name=juror["name"], style=juror["style"],
            pro=prosecution, def_=defense, wit=witness,
            hist=snippet, r=rnd, R=ROUNDS))
        tag = f"{juror['name']}: "
        history.append(tag + speech)
        print("\n" + tag + speech + "\n")

print("\n=== Final Votes ===")
votes: Dict[str, str] = {}
full_hist = "\n".join(history)
for juror in JURORS:
    vtxt = chat(VOTE_TMPL.format(name=juror["name"], full=full_hist), temp=0.7)
    verdict = ("Not Guilty" if "Not Guilty" in vtxt
               else "Guilty" if "Guilty" in vtxt
               else "Abstain")
    votes[juror["name"]] = verdict
    print(f"{juror['name']}: {verdict}")

g  = list(votes.values()).count("Guilty")
ng = list(votes.values()).count("Not Guilty")
ab = list(votes.values()).count("Abstain")
print(f"\nTally → Guilty: {g}, Not Guilty: {ng}, Abstain: {ab}")
if g > len(JURORS)/2:   print("🧑‍⚖️ Verdict: DEFENDANT LIABLE")
elif ng > len(JURORS)/2:print("🧑‍⚖️ Verdict: DEFENDANT JUSTIFIED")
else:                   print("🧑‍⚖️ Verdict: HUNG JURY")
